In [6]:
# ============================================================
# CORPORATE INTEREST & OTHER INCOME — Calibrated Simple Model
# ============================================================

import pandas as pd

# ------------------------------------------------------------
# 1. INPUT DATA — Fill with real quarterly values
# ------------------------------------------------------------
# All monetary values in billions unless noted otherwise

data = pd.DataFrame({
    "Quarter": ["2024Q1", "2024Q2", "2024Q3", "2024Q4", "2025Q1", "2025Q2"],

    # Average Coinbase corporate cash + treasury holdings (billions)
    # → From 10-Q "Cash and cash equivalents" + "Treasury investments"
    "cash_treasuries_bil": [6.7, 7.2, 7.7, 8.5, 7.5, 8.1],

    # Quarterly average Fed Funds rate (%)
    # → From FRED, CME, or other macro data source
    "fed_rate": [5.33, 5.33, 5.26, 4.65, 4.33, 4.33],

    # Reported “Corporate interest and other income” in millions
    # → From Coinbase income statement
    "reported_mil": [49.9, 69.7, 76.6, 74.6, 74.0, 77.1],
})

# ------------------------------------------------------------
# 2. CALIBRATE THE MULTIPLIER FROM HISTORY
# ------------------------------------------------------------
# This absorbs all conversion, utilization, and yield effects.
# Formula: reported = (cash * rate) * k  →  k = Σ(reported) / Σ(cash*rate)

data["cash_rate_product"] = data["cash_treasuries_bil"] * data["fed_rate"]
k = data["reported_mil"].sum() / data["cash_rate_product"].sum()

print(f"Calibrated factor k = {k:.3f}  (units: M / (B * %))")

# ------------------------------------------------------------
# 3. COMPUTE MODELED VALUES
# ------------------------------------------------------------
data["modeled_mil"] = data["cash_rate_product"] * k
data["error_mil"] = data["reported_mil"] - data["modeled_mil"]
data["pct_error"] = 100 * data["error_mil"] / data["reported_mil"]

print("\nBacktest results:")
print(data[["Quarter", "reported_mil", "modeled_mil", "pct_error"]])

avg_err = abs(data["pct_error"]).mean()
print(f"\nAverage absolute percent error: {avg_err:.1f}%")

# ------------------------------------------------------------
# 4. FORECAST NEXT QUARTER (EXAMPLE: 2025Q3)
# ------------------------------------------------------------
# Replace with your assumptions
q3_cash_treasuries_bil = 6.8    # Expected average cash + treasuries ($B)
q3_fed_rate = 5.33              # Expected average Fed Funds rate (%)

q3_forecast_mil = q3_cash_treasuries_bil * q3_fed_rate * k

print(f"\nPredicted 2025Q3 Corporate Interest & Other Income: ${q3_forecast_mil:,.1f} million")

# ------------------------------------------------------------
# Notes:
# ------------------------------------------------------------
# - 'k' automatically captures Coinbase’s effective yield,
#   utilization, and unit conversion to millions.
# - As you add new quarters, re-run to update 'k'.
# - If the factor drifts over time, consider separate k_highrate / k_lowrate regimes.


Calibrated factor k = 1.903  (units: M / (B * %))

Backtest results:
  Quarter  reported_mil  modeled_mil  pct_error
0  2024Q1          49.9    67.970473 -36.213372
1  2024Q2          69.7    73.042896  -4.796120
2  2024Q3          76.6    77.089415  -0.638922
3  2024Q4          74.6    75.229843  -0.844294
4  2025Q1          74.0    61.811237  16.471301
5  2025Q2          77.1    66.756136  13.416166

Average absolute percent error: 12.1%

Predicted 2025Q3 Corporate Interest & Other Income: $69.0 million
